In [1]:
from schema import *

s = Schema()

# Load up the CSVs
cardnum = pd.read_csv("./csv/bonuses/cardnum.csv").set_index("val_id")
person = pd.read_csv("./csv/bonuses/person.csv").set_index("cardnum")

# Insert them into the data frames
s.insert_dataframe(cardnum, "cardnum")
s.insert_dataframe(person, "person")

# Get handles on the nodes now in the schema - may be slightly ugly but
# we can slap on a load of syntactic sugar
c_cardnum = SchemaNode("cardnum", cluster="cardnum")
c_val_id = SchemaNode("val_id", cluster="cardnum")

p_cardnum = SchemaNode("cardnum", cluster="person")
p_person = SchemaNode("person", cluster="person")

s.blend(c_cardnum, p_cardnum, under="Cardnum")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
from tables import *
table1 = s.get(['cardnum.val_id'])
print(table1)

[cardnum.val_id || ]
Empty DataFrame
Columns: []
Index: [1, 2, 3, 4, 5, 8]


In [3]:
table2 = table1.infer(['cardnum.val_id'], 'cardnum.cardnum')
print(table2)

[cardnum.val_id || cardnum.cardnum]
                cardnum.cardnum
cardnum.val_id                 
1                          5172
2                          2354
3                          1410
4                          1111
5                          2354
8                          4412


In [4]:
table3 = table2.infer(['cardnum.cardnum'], 'person.cardnum')
print(table3)

[cardnum.val_id || cardnum.cardnum person.cardnum]
                cardnum.cardnum  person.cardnum
cardnum.val_id                                 
1                          5172            5172
2                          2354            2354
5                          2354            2354
3                          1410            1410
4                          1111            1111
8                          4412            4412


In [5]:
table4 = table3.infer(['person.cardnum'], 'person.person')
print(table4)

[cardnum.val_id || cardnum.cardnum person.cardnum person.person]
                cardnum.cardnum  person.cardnum person.person
cardnum.val_id                                               
1.0                      5172.0            5172           NaN
2.0                      2354.0            2354         Steve
5.0                      2354.0            2354         Steve
3.0                      1410.0            1410           Tom
4.0                      1111.0            1111         Steve
8.0                      4412.0            4412           NaN
4 keys hidden


In [6]:
table = table4.hide('person.cardnum').hide('cardnum.cardnum')
print(table)

[cardnum.val_id || person.person]
               person.person
cardnum.val_id              
2.0                    Steve
5.0                    Steve
3.0                      Tom
4.0                    Steve
2 keys hidden
2 values hidden


In [8]:
table1 = s.get(['cardnum.val_id'])
print(table1)

[cardnum.val_id || ]
Empty DataFrame
Columns: []
Index: [1, 2, 3, 4, 5, 8]


In [9]:
table1.infer(['cardnum.val_id'], 'person.person')

[cardnum.val_id || person.person]
               person.person
cardnum.val_id              
2.0                    Steve
3.0                      Tom
4.0                    Steve
5.0                    Steve
2 keys hidden
2 values hidden


In [10]:
table = (s.get(['person.cardnum']).infer(['person.cardnum'], 'person.person'))
print(table)

[person.cardnum || person.person]
               person.person
person.cardnum              
1111                   Steve
1410                     Tom
2354                   Steve
6440                   Harry
5467                    Dick


In [16]:
tbl1 = table.compose(['cardnum.val_id'], 'person.cardnum')
print(table1)

table2 = tbl1.set_key(["person.person"])


AssertionError: 

In [ ]:
table1 = table

In [12]:
table = s.get(['person.person']).infer(['person.person'], 'person.cardnum')
print(table)

person.cardnum
[person.person || person.cardnum]
              person.cardnum
person.person               
Steve           [1111, 2354]
Tom                   [1410]
Harry                 [6440]
Dick                  [5467]


In [ ]:
table1 = table.show('person.cardnum')
print(table1)
table2 = table1.hide('person.cardnum')
print(table2)